In [9]:
from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import CovarianceShrinkage
import cvxpy as cp
import numpy as np
import pandas as pd
import torch

In [10]:
df = pd.read_csv("stock_prices.csv").drop("date", axis=1)

In [11]:
sector_mapper = {
        "GOOG": "tech",
        "AAPL": "tech",
        "FB": "tech",
        "AMZN": "tech",
        "BABA": "tech",
        "GE": "utility",
        "AMD": "tech",
        "WMT": "retail",
        "BAC": "fig",
        "GM": "auto",
        "T": "auto",
        "UAA": "airline",
        "SHLD": "retail",
        "XOM": "energy",
        "RRC": "energy",
        "BBY": "retail",
        "MA": "fig",
        "PFE": "pharma",
        "JPM": "fig",
        "SBUX": "retail",
    }


In [12]:
mu = mean_historical_return(df)
S = CovarianceShrinkage(df).ledoit_wolf()

In [18]:
S

,GOOG,AAPL,FB,BABA,AMZN,GE,AMD,WMT,BAC,GM,T,UAA,SHLD,XOM,RRC,BBY,MA,PFE,JPM,SBUX
GOOG,0.045529,0.022143,0.006389,0.003720,0.026085,0.015815,0.021761,0.008238,0.026957,0.006163,0.011058,0.021371,0.016418,0.013362,0.018714,0.016089,0.019593,0.010683,0.023249,0.018075
AAPL,0.022143,0.207037,0.004334,0.002954,0.058200,0.038102,0.084053,0.026429,0.045966,0.005791,0.019846,0.020659,0.022127,0.020236,0.029179,0.048719,0.019587,0.021324,0.046814,0.039789
FB,0.006389,0.004334,0.029233,0.003770,0.007619,0.003008,0.005804,0.001243,0.004766,0.003499,0.001395,0.009407,0.002191,0.002214,0.002741,0.003813,0.004727,0.002550,0.003842,0.004501
BABA,0.003720,0.002954,0.003770,0.013438,0.004176,0.002011,0.006332,0.001081,0.003077,0.002573,0.000891,0.003461,0.003114,0.001751,0.002973,0.002441,0.003182,0.001624,0.002654,0.002162
AMZN,0.026085,0.058200,0.007619,0.004176,0.276365,0.038169,0.075657,0.027457,0.049062,0.006527,0.018720,0.027889,0.027333,0.019637,0.029789,0.059925,0.023092,0.024170,0.053367,0.048322
GE,0.015815,0.038102,0.003008,0.002011,0.038169,0.083405,0.048580,0.029076,0.062871,0.007533,0.026464,0.025245,0.025666,0.025815,0.029488,0.041375,0.019308,0.029891,0.059131,0.034449
AMD,0.021761,0.084053,0.005804,0.006332,0.075657,0.048580,0.388916,0.027113,0.068272,0.011758,0.026781,0.030291,0.035381,0.027946,0.049354,0.066146,0.026956,0.027965,0.069592,0.051764
WMT,0.008238,0.026429,0.001243,0.001081,0.027457,0.029076,0.027113,0.069478,0.030560,0.003595,0.020985,0.012137,0.015605,0.017036,0.012011,0.037385,0.009599,0.023160,0.033017,0.025482
BAC,0.026957,0.045966,0.004766,0.003077,0.049062,0.062871,0.068272,0.030560,0.179868,0.012581,0.033848,0.040611,0.039452,0.032159,0.045077,0.052162,0.035707,0.034984,0.116165,0.047582
GM,0.006163,0.005791,0.003499,0.002573,0.006527,0.007533,0.011758,0.003595,0.012581,0.021594,0.003942,0.009720,0.011321,0.005865,0.007549,0.007759,0.007317,0.004823,0.010176,0.006414


In [13]:

ds = []
for i in range(10000):
	max_tech = np.random.rand()
	max_retail = np.random.rand()
	max_fig = np.random.rand()
	max_energy = np.random.rand()
	utility_all = 0.2*np.random.rand()
	airline_all = 0.2*np.random.rand()
	rho = (10 - 1)*np.random.rand() + 1
	
	weights = cp.Variable(len(mu))
	constraints = [
		weights >= 0.0,
		cp.sum(weights) == 1.0,
		weights[5] == utility_all,
		weights[11] == airline_all,
		weights[[0,1,2,3,4,6]] <= max_tech,
		weights[[7,12,15,19]] <= max_retail,
		weights[[8,16,18]] <= max_fig,
		weights[[13,14]] <= max_energy
	]

	prob = cp.Problem(cp.Minimize(-(weights.T @ mu - 0.5 * rho* cp.quad_form(weights, S))),constraints)
	prob.solve()
	if prob.status == 'optimal': 
		X = np.array([max_tech, max_retail, max_fig, max_energy, utility_all, airline_all, rho])
		#y = np.array([w for w in list(weights.value)])
		ds.append((X, weights.value))

In [23]:
in_dims = [0]

In [24]:
in_dims.append({'x': 0})

In [26]:
dict = {'x': 0, 'b': 5}

In [27]:
batched_dict = {}
for param in dict:
    batched_dict[param] = 0

In [29]:
batched_dict

{'x': 0, 'b': 0}

In [14]:
[ds[i][1] for i in range(len(ds))]

[array([ 1.94134676e-02,  2.51683539e-02,  4.53836522e-01,  1.57471527e-01,
         4.62176047e-02,  4.14204674e-02, -3.65487222e-23,  8.39352088e-23,
         4.29365367e-24, -1.95883375e-23,  5.29637043e-23,  1.09243248e-01,
         1.17275501e-23,  3.75139193e-23, -5.43513063e-24,  1.03981680e-24,
         1.30581250e-01,  1.66475590e-02,  5.06052883e-25,  2.94780199e-23]),
 array([-4.17358860e-22,  6.67083801e-03,  4.81527474e-01, -2.94034221e-22,
         4.94605019e-02,  1.58121912e-01, -4.31562532e-23, -2.09727830e-22,
        -7.86284427e-23, -2.48260320e-22, -2.40448696e-22,  6.66945971e-02,
        -9.04659696e-23, -2.99050441e-22, -6.18579643e-23, -4.22019402e-23,
         2.37524677e-01, -4.12414407e-23, -2.01527977e-22, -4.19739669e-23]),
 array([ 2.74411207e-02,  2.45079466e-02,  2.66713941e-01,  1.30479534e-01,
         6.93230122e-02,  1.40635667e-01, -1.02928708e-24, -6.19628044e-23,
        -6.57127130e-23, -7.57375196e-23, -9.95199790e-23,  9.61608647e-02,
        

In [15]:
ds[0][1][[0,1,2,3,4,6]].sum()

0.7021074754307454

In [16]:
from pickle import dump
dump(ds, open("portfolio.pkl", "wb"))